In [1]:
import torch.nn as nn
import torch.nn.functional as F

# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)  
#         self.fc1 = nn.Linear(32 * 14 * 14, 128)  
#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)  
#         x = x.view(-1, 32 * 14 * 14) 
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.1)  
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc2(x)
        return x


In [2]:
# train
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# test_loader = DataLoader(
#     datasets.MNIST('./data', train=True, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=64, shuffle=True)

# 数据加载和预处理
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

train_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=True, download=True, transform=transforms_train),
    batch_size=64, shuffle=True)

# 模型实例化和优化器设置
model = SimpleCNN()
optimizer = optim.Adamax(model.parameters(), lr=0.01)

# 设备设置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

# 训练过程
for epoch in range(1, 30):
    for batch_idx, (data, target) in enumerate(train_loader):
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# # 保存模型
# torch.save(model.state_dict(), f'./model/mnist_cnn.pt')

# 保存模型
torch.save(model.state_dict(), './model/cifar10_cnn.pt')



Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308306
Train Epoch: 1 [10240/50000 (20%)]	Loss: 3.909317
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.184709
Train Epoch: 1 [30720/50000 (61%)]	Loss: 2.155690
Train Epoch: 1 [40960/50000 (82%)]	Loss: 2.102313
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.068285
Train Epoch: 2 [10240/50000 (20%)]	Loss: 1.975598
Train Epoch: 2 [20480/50000 (41%)]	Loss: 1.926683
Train Epoch: 2 [30720/50000 (61%)]	Loss: 1.920215
Train Epoch: 2 [40960/50000 (82%)]	Loss: 1.894750
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.847780
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1.860749
Train Epoch: 3 [20480/50000 (41%)]	Loss: 1.788023
Train Epoch: 3 [30720/50000 (61%)]	Loss: 1.751833
Train Epoch: 3 [40960/50000 (82%)]	Loss: 1.779747
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.739079
Train Epoch: 4 [10240/50000 (20%)]	Loss: 1.661044
Train Epoch: 4 [20480/50000 (41%)]	Loss: 1.663636
Train Epoch: 4 [30720/50000 (61%)]	Loss: 1.646841
Train Epoch: 4 [40960/50000 (82%

In [3]:
# test.py
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# test_loader = DataLoader(
#     datasets.MNIST('./data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=1000, shuffle=True)


# 数据加载
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

test_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=False, download=True, transform=transforms_train),
    batch_size=1000, shuffle=True)

model = SimpleCNN()
# model.load_state_dict(torch.load('./model/mnist_cnn.pt'))
model.load_state_dict(torch.load('./model/cifar10_cnn.pt'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')




Files already downloaded and verified

Test set: Average loss: 0.9623, Accuracy: 6632/10000 (66%)

